In [1]:
import os
owd = os.getcwd()

In [2]:
os.chdir(owd)
import numpy as np
import re
import pandas as pd
import ipywidgets as wdg
from plotly_default import go
from TecplotTools import read_tecplot, data_cats, plot_var_range, plot_bounds

In [3]:
# Import the plot categories
os.chdir(owd)
plot_cats = pd.read_csv('./plot_categories.csv', keep_default_na=False, delimiter=';')

# Get list of tec files (note that this changes the cwd)
catList, max_time = data_cats(r"D:/Peter/Code/CrunchTope/CS_Link")

# Get the vertical scale
with open(r'CSColumnShortARSR.in') as f:
   all_lines = f.read()
   distance_units = re.search('distance_units +(\w+)', all_lines).group(1)
   x_res = re.search('xzones +(\d+) (\d+)', all_lines).group(2)

# filter categories
# plot_cats.query('file_name in @catList & ignore == False', inplace=True)
plot_cats.query('ignore == False', inplace=True)
plot_cats.loc[plot_cats['file_name'] == 'depth', 'units'] = distance_units
plot_cats

,file_name,friendly_name,prefix,suffix,units,starts_log,ignore
0,depth,Depth,,<i>z</i>,centimeters,False,False
1,AqRate,Aqueous reaction rate,Rate of,,mol dm<sup>−3</sup> s<sup>−1</sup>,False,False
2,TotMineral,Component concentration in minerals,Concentration of,in minerals,mol kg<sup>−1</sup>,False,False
3,affinity_term,Monod affinity term,Affinity term for,,,False,False
4,aq_affinity,Monod biomass aqueous affinity term,Affinity term for,,,False,False
5,area,Mineral surface area,Surface area of,,m<sup>2</sup>,False,False
6,conc,Aqueous species concentration,Concentration of,,mol kg<sup>−1</sup>,True,False
7,pH,pH,,,,False,False
8,porosity,Porosity,,,,False,False
9,rate,Mineral precipitation rate,Rate of,precipitation,mol m<sup>−3</sup> s<sup>−1</sup>,False,False


In [7]:
def get_filename(x_var_type):
    return str(plot_cats.query(f"friendly_name == '{x_var_type.value}'")['file_name'].iloc[0])

# Initialise widgets
play = wdg.Play(value=1, min=1, max=max_time, step=1, interval=1)
time_slider = wdg.IntSlider(value=1, min=1, max=max_time, step=1, layout=wdg.Layout(width='50%'), description='Time')
wdg.jslink((play, 'value'), (time_slider, 'value'))

x_var_type = wdg.Dropdown(value='Component concentration', options=plot_cats['friendly_name'], description='X axis:')
x_var = wdg.Dropdown(options=(read_tecplot(get_filename(x_var_type), time_slider.value)[1])[3:])
x_log = wdg.Checkbox(value=False, description='log')
x_rev = wdg.Checkbox(value=False, description='Reverse')

y_var_type = wdg.Dropdown(value='Depth', options=plot_cats['friendly_name'], description='Y axis:')
y_var = wdg.Dropdown(options=['Depth,'])
y_log = wdg.Checkbox(value=False, description='log')
y_rev = wdg.Checkbox(value=True, description='Reverse')


def initialise1D(file_cat='totcon'):
    df, column_headers = read_tecplot(file_cat, 1)
    # df.eval('X_scaled = X * @x_res', inplace=True)
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(y=np.zeros_like(df["X"]), x=np.zeros_like(df["X"])))
    line = fig.select_traces().__next__()
    fig.update_layout(
        yaxis=dict(autorange="reversed", rangemode="tozero", title=f"Depth, <i>z</i> ({distance_units})"),
        hovermode="y",
        width=600,
        height=700,
    )
    return fig, line

def update_var_list(*args):
    if get_filename(x_var_type) == 'depth':
        x_var.options = ['Depth,']
    else:
        x_vars_list = read_tecplot(get_filename(x_var_type), time_slider.value)[1]
        x_var.options = x_vars_list[3:]

    if get_filename(y_var_type) == 'depth':
        y_var.options = ['Depth,']
    else:
        y_vars_list = read_tecplot(get_filename(y_var_type), time_slider.value)[1]
        y_var.options = y_vars_list[3:]
        

def update_plot(time, plot_var_type, plot_var, plot_log, plot_rev, axis):
    plot_info = plot_cats.query(f"friendly_name == '{plot_var_type}'")
    
    file_cat = plot_info['file_name'].iloc[0]
    pref_str = plot_info['prefix'].iloc[0]
    suff_str = plot_info['suffix'].iloc[0]
    unit_str = plot_info['units'].iloc[0]
    if unit_str != '':
        unit_str = '(' + unit_str + ')'
    starts_log = bool(plot_info['starts_log'].iloc[0])
    
    
    # Get variable to plot on x axis
    if file_cat == 'depth':
        df, column_headers = read_tecplot('totcon', time)
        df.eval('X_scaled = X * @x_res', inplace=True)
        xx = np.array(df['X_scaled'])
    else:
        df, column_headers = read_tecplot(file_cat, time)
        xx = np.array(df[plot_var])
    
    # Correct log values
    if plot_log == False:
        if starts_log == True:
            xx = 10**xx
        else:
            pass
    else:
        pref_str = 'log<sub>10</sub> ' + pref_str
        if starts_log == True:
            pass
        else:
            xx = np.log10(xx)
            
    
    # lower, upper =  plot_var_range(max_time, file_cat, plot_var)
    lower = np.min(xx)
    upper = np.max(xx)
    lower, upper = plot_bounds(lower, upper)
    
    # swap bounds for reversed axis
    if plot_rev:
        lower, upper = upper, lower
    
    with fig.batch_update():
        if axis == 'x':
            line.x = xx
            fig.update_layout(xaxis_range = [lower, upper],
                              xaxis_title = f'{pref_str} {plot_var} {suff_str}{unit_str}')
        elif axis == 'y':
            line.y = xx
            fig.update_layout(yaxis_range = [lower, upper],
                              yaxis_title = f'{pref_str} {plot_var} {suff_str}{unit_str}')
        else:
            raise ValueError("Please enter 'x' or 'y' for the axis")

x_var_type.observe(update_var_list, 'value')
y_var_type.observe(update_var_list, 'value')

fig, line = initialise1D('totcon')

out1 = wdg.interactive_output(update_plot, dict(plot_var_type=x_var_type, time=time_slider, plot_var=x_var, plot_log=x_log, plot_rev=x_rev, axis=wdg.fixed('x')))
out2 = wdg.interactive_output(update_plot, dict(plot_var_type=y_var_type, time=time_slider, plot_var=y_var, plot_log=y_log, plot_rev=y_rev, axis=wdg.fixed('y')))

# Display widgets
wdg.VBox([
    wdg.HBox([x_var_type, x_var, x_log, x_rev]),
    wdg.HBox([y_var_type, y_var, y_log, y_rev]),
    wdg.HBox([play, time_slider]),
    wdg.HBox([fig]),
    out1,
    out2
])